In [ ]:
import socket
import struct
import time
from pyModeS.decoder.bds import bds

BEAST_IP = '129.123.91.145'
BEAST_PORT = 30005

def read_beast_messages(duration=10):
    """Connect to Beast port and read binary data for a given duration (seconds)."""
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((BEAST_IP, BEAST_PORT))
    s.settimeout(2)

    start_time = time.time()
    buffer = b""

    messages = []

    while time.time() - start_time < duration:
        try:
            buffer += s.recv(4096)
        except socket.timeout:
            break

        while b'\x1a' in buffer:
            start = buffer.index(b'\x1a')
            if len(buffer) < start + 2:
                break

            msg_type = buffer[start + 1]
            if msg_type != 0x31:  # Only handle Mode-S extended
                buffer = buffer[start + 1:]
                continue

            if len(buffer) < start + 3 + 6 + 1:  # Header + ts + length
                break

            ts_offset = start + 2
            length_offset = ts_offset + 6
            msg_len = buffer[length_offset]
            full_len = length_offset + 1 + msg_len

            if len(buffer) < full_len:
                break

            payload = buffer[length_offset + 1: full_len]
            hex_msg = payload.hex().upper()

            timestamp = int.from_bytes(buffer[ts_offset:ts_offset + 6], 'big')
            messages.append((timestamp, hex_msg))

            buffer = buffer[full_len:]

    s.close()
    return messages

def decode_message(hex_msg):
    """Decode Mode-S hex message using pyModeS."""
    result = {}
    try:
        if not pyModeS.crc.check(hex_msg):
            return None  # discard messages with bad CRC

        df = pyModeS.df(hex_msg)
        result["df"] = df
        result["icao"] = pyModeS.icao(hex_msg)

        if df == 17:  # ADS-B messages
            tc = pyModeS.adsb.typecode(hex_msg)
            result["typecode"] = tc

            if 1 <= tc <= 4:
                result["callsign"] = pyModeS.adsb.callsign(hex_msg)
            elif 9 <= tc <= 18:
                position = pyModeS.adsb.position_with_ref(hex_msg, None, None)  # Needs reference for lat/lon
                result["position"] = position
                result["altitude"] = pyModeS.adsb.altitude(hex_msg)
            elif 19 <= tc <= 22:
                result["velocity"] = pyModeS.adsb.velocity(hex_msg)

    except Exception as e:
        result["error"] = str(e)

    return result

# Run it
if __name__ == "__main__":
    print("Connecting to Beast stream...")
    raw_msgs = read_beast_messages(duration=10)
    print(f"Captured {len(raw_msgs)} messages.")

    print("\nDecoded examples:")
    for ts, msg in raw_msgs[:10]:  # show first 10 decoded
        decoded = decode_message(msg)
        if decoded:
            print(f"\nTimestamp: {ts}")
            print(f"Hex: {msg}")
            for k, v in decoded.items():
                print(f"  {k}: {v}")